In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"


In [2]:
from pathlib import Path

In [3]:
import os, json, cv2, numpy as np
from pathlib import Path
from tqdm.auto import tqdm
import albumentations as A

In [ ]:
# #original run
# !python ../src/train_faster_rcnn.py \
#  --train-json ../artifacts/train.json \
#  --val-json ../artifacts/val.json \
#  --epochs 30 \
#  --profile gpu \
#  --img-size 640 \
#  --batch-size 2 \
#  --train-aug light \
#  --freeze-up-to 2 \
#  --lr 0.002 \
#  --lr-backbone 0.0002 \
#  --num-workers 0 \
#  --scheduler none

^C


# RUN THIS

In [40]:
# prepare the data
!python src/faster_rcnn/data/prepare_data_faster_rcnn.py \
    --datasets src/faster_rcnn/data/center_stratified/C3 \
    --out-dir src/artifacts \
    --class-name polyp


[info] processing dataset: /Users/rebekaheichberg/Desktop/git-re/colorectal_imaging/Architectures/src/faster_rcnn/data/center_stratified/C3
[info] detected layout A under src/faster_rcnn/data/center_stratified/C3
   train images: True → src/faster_rcnn/data/center_stratified/C3/images/train
   train labels: True → src/faster_rcnn/data/center_stratified/C3/labels/train
   val images:   True → src/faster_rcnn/data/center_stratified/C3/images/val
   val labels:   True → src/faster_rcnn/data/center_stratified/C3/labels/val
[info] scanning images in src/faster_rcnn/data/center_stratified/C3/images/train
[info] found 314 images in src/faster_rcnn/data/center_stratified/C3/images/train
[diag] src/faster_rcnn/data/center_stratified/C3/images/train: images=314 matched_labels=314 boxes=368 skipped=0
[info] scanning images in src/faster_rcnn/data/center_stratified/C3/images/val
[info] found 39 images in src/faster_rcnn/data/center_stratified/C3/images/val
[diag] src/faster_rcnn/data/center_strati

In [41]:
%cd /Users/rebekaheichberg/Desktop/git-re/colorectal_imaging/Architectures
%env PYTHONPATH=src:$PYTHONPATH

!python -m faster_rcnn.training.train_faster_rcnn \
    --train-json src/artifacts/train.json \
    --val-json src/artifacts/val.json \
    --roots-map src/artifacts/roots_map.json \
    --images-root src/faster_rcnn/data/center_stratified/C3 \
    --epochs 20 \
    --batch-size 8 \
    --num-workers 0 \
    --img-size 640 \
    --train-augs medium \
    --num-classes 2 \
    --freeze-backbone 2 \
    --opt sgd \
    --lr-backbone 1e-4 \
    --lr-heads 5e-3 \
    --weight-decay 1e-4 \
    --lr-scheduler none \
    --device cpu
    

/Users/rebekaheichberg/Desktop/git-re/colorectal_imaging/Architectures
env: PYTHONPATH=src:$PYTHONPATH
/Users/rebekaheichberg/Desktop/git-re/colorectal_imaging/Architectures/src/faster_rcnn/augmentations/presets_faster_rcnn.py:312: UserWarning: Argument(s) 'cval' are not valid for transform Affine
  A.Affine(
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
/Users/rebekaheichberg/Desktop/git-re/colorectal_imaging/Architectures/src/faster_rcnn/training/train_faster_rcnn.py:255: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device.type == "cuda"))
Train E1:   0%|                                                              | 0/40 [00:00<?, ?it/s]/Users/rebekaheichberg/anaconda3/envs/erdos_summer_2025/lib/python3.12/site-packages/torch/utils/data/da

# STOP HERE FOR INFERENCE

In [4]:
import json
from pathlib import Path

root = Path("/Users/rebekaheichberg/Desktop/git-re/colorectal_imaging/Architectures")
val_json_path   = root / "src" / "artifacts" / "val.json"

with val_json_path.open("r") as f:
    data = json.load(f)

print("keys BEFORE:", data.keys())

# Add missing keys for pycocotools
if "info" not in data:
    data["info"] = {"description": "polyp dataset"}
if "licenses" not in data:
    data["licenses"] = []

with val_json_path.open("w") as f:
    json.dump(data, f)

print("keys AFTER:", data.keys())


keys BEFORE: dict_keys(['images', 'annotations', 'categories', 'info', 'licenses'])
keys AFTER: dict_keys(['images', 'annotations', 'categories', 'info', 'licenses'])


In [12]:
#!/usr/bin/env python3
from pathlib import Path
import torch
import sys
from pathlib import Path

# Add your src folder to Python path
SRC = Path("/Users/rebekaheichberg/Desktop/git-re/colorectal_imaging/Architectures/src")
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))


from faster_rcnn.data.data_loaders_faster_rcnn import build_train_val_loaders
from faster_rcnn.data.coco_eval_faster_rcnn import coco_map
from faster_rcnn.models.faster_rcnn_model import build_fasterrcnn

# --- paths ---
root = Path("/Users/rebekaheichberg/Desktop/git-re/colorectal_imaging/Architectures")
ckpt_path = root / "runs" / "frcnn_polyp" /"epoch_009.pth"
train_json = root / "src" / "artifacts" / "train.json"
val_json   = root / "src" / "artifacts" / "val.json"
roots_map  = root / "src" / "artifacts" / "roots_map.json"
images_root = root / "faster_rcnn" / "data" / "center_stratified" / "C3" / "images" 
# --- setup model ---
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = build_fasterrcnn(num_classes=2, pretrained=False)
state = torch.load(ckpt_path, map_location=device)
model.load_state_dict(state["model"])
model.to(device).eval()

# --- build validation loader ---
# (these args match your build_train_val_loaders signature)
_, val_loader = build_train_val_loaders(
    train_json=str(train_json),
    val_json=str(val_json),
    roots_map=str(roots_map),
    images_root=str(images_root),
    img_size=832,         # match your training size
    train_augs="none",    # val loader ignores this anyway
    batch_size=2,
    num_workers=0,
)

# --- run COCO eval ---
stats = coco_map(model, val_loader, device, coco_gt_json=str(val_json))
print(f"COCO AP={stats[0]:.4f}, AP50={stats[1]:.4f}, AP75={stats[2]:.4f}")


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


COCO eval:   0%|                                                             | 0/20 [00:00<?, ?it/s]/Users/rebekaheichberg/anaconda3/envs/erdos_summer_2025/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
COCO eval: 100%|█████████████████████████████████████████████████| 20/20 [1:35:08<00:00, 285.44s/it]

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.001
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.016
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

In [1]:
from pathlib import Path

root = Path("/Users/rebekaheichberg/Desktop/git-re/colorectal_imaging/Architectures")  # or whatever it is

ckpt = root / "runs" / "frcnn_polyp" / "epoch_009.pth"
images_dir = root / "src" / "faster_rcnn" / "data" / "center_stratified" / "C3" / "images" / "val" 
out_dir = root / "runs" /"frcnn_polyp" / "inference_side_by_side_rebekah"


In [8]:
%cd /Users/rebekaheichberg/Desktop/git-re/colorectal_imaging/Architectures

!python src/faster_rcnn/inference/infer_folder_faster_rcnn.py \
    --weights "{ckpt}" \
    --images-dir "{images_dir}" \
    --out-dir "{out_dir}" \
    --coco-json src/artifacts/val.json \
    --num-classes 2 \
    --score-thr 0.5 \
    --nms-iou 0.4 \
    --debug-gt


/Users/rebekaheichberg/Desktop/git-re/colorectal_imaging/Architectures
[info] using device: cpu
[info] building model...
[info] weights loaded.
[info] loading COCO GT from src/artifacts/val.json
[info] found 39 images.
[1/39] c3s000p4_0d17_C3_EndoCV2021_00180.jpg
    GT annos: 4
[2/39] c3s001p1_24d7_C3_EndoCV2021_00150.jpg
    GT annos: 1
[3/39] c3s002p5_0d18_C3_EndoCV2021_00369.jpg
    GT annos: 5
[4/39] c3s003p1_2d63_C3_EndoCV2021_00167.jpg
    GT annos: 1
[5/39] c3s004p1_23d2_C3_EndoCV2021_00245.jpg
    GT annos: 1
[6/39] c3s005p1_63d8_C3_EndoCV2021_00275.jpg
    GT annos: 1
[7/39] c3s006p1_1d58_C3_EndoCV2021_00139.jpg
    GT annos: 1
[8/39] c3s007p1_35d3_C3_EndoCV2021_00233.jpg
    GT annos: 1
[9/39] c3s008p1_27d8_C3_EndoCV2021_00327.jpg
    GT annos: 1
[10/39] c3s009p1_21d5_C3_EndoCV2021_00119.jpg
    GT annos: 1
[11/39] c3s010p2_4d01_C3_EndoCV2021_00142.jpg
    GT annos: 2
[12/39] c3s011p1_16d0_C3_EndoCV2021_00101.jpg
    GT annos: 1
[13/39] c3s012p1_5d14_C3_EndoCV2021_00371.jpg
